![Numpy Slide](img/01_Numpy/Folie1.PNG)

![Numpy Slide](img/01_Numpy/Folie2.PNG)

![Numpy Slide](img/01_Numpy/Folie3.PNG)

![Numpy Slide](img/01_Numpy/Folie4.PNG)

In [1]:
import numpy as np
import time
import sys

# Setup for performance comparison
size = 1_000_000
python_list = list(range(size))
numpy_array = np.arange(size)

print(f"Array size: {size} elements")
print(f"Expected size (8 bytes per element): {8 * size} bytes")

print(f"Python list memory: {sys.getsizeof(python_list)} bytes. (Overhead: {sys.getsizeof(python_list) - (8 * size)} bytes)")
print(f"NumPy sys memory: {sys.getsizeof(numpy_array)} bytes. (Overhead: {sys.getsizeof(numpy_array) - (8 * size)} bytes)")

Array size: 1000000 elements
Expected size (8 bytes per element): 8000000 bytes
Python list memory: 8000056 bytes. (Overhead: 56 bytes)
NumPy sys memory: 8000112 bytes. (Overhead: 112 bytes)


As you can see, each element takes 8 bytes with the python list having a very tiny overhead of 56 bytes compared to the overhead of the NumPy array which clocks in at 112 bytes. Still, both overheads are negligible when working with big arrays. 

> *Note*: You can also get the size (in bytes) of the numpy array without the overhead by using the `.nbytes` property

Next, we compare a basic multiplication on the list compared to the numpy array:

In [2]:
# Performance comparison: Element-wise multiplication
def python_multiply(data):
    return [x * 2 for x in data]

def numpy_multiply(data):
    return data * 2

# Time Python approach
start = time.time()
python_result = python_multiply(python_list)
python_time = time.time() - start

# Time NumPy approach
start = time.time()
numpy_result = numpy_multiply(numpy_array)
numpy_time = time.time() - start

print(f"Python list comprehension: {python_time:.4f} seconds")
print(f"NumPy vectorized operation: {numpy_time:.4f} seconds")
print(f"NumPy is {python_time / numpy_time:.1f}x faster!")

Python list comprehension: 0.0376 seconds
NumPy vectorized operation: 0.0025 seconds
NumPy is 14.9x faster!


Of course, that's technically not the best way to measure performance. Measuring performance should be done with a dedicated library that measures the same code multiple times and averages the runtime while also reporting the standard deviation. For Python, one such library is `timeit`:

In [3]:
%timeit python_multiply(python_list)
%timeit numpy_multiply(numpy_array)

33.9 ms ± 709 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
525 μs ± 19.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Now we can say with confidence, that numpy is much, much faster than raw python!

## 1.2 NumPy Array Internals

Understanding NumPy's internal structure is crucial for writing high-performance code.

### The `ndarray` Structure

Every NumPy array consists of:
1. **Data buffer**: Raw binary data
2. **Metadata**: Shape, strides, dtype, etc.
3. **View information**: How to interpret the data buffer

![Numpy Slide](img/01_Numpy/Folie5.PNG)

In [4]:
# Create a 2D array to explore its internals
x = np.array([[0, 1, 2, 3],
              [4, 5, 6, 7],
              [8, 9, 10, 11],
              [12, 13, 14, 15]], dtype=np.int8)

print("Array:")
print(x)
print(f"\nShape: {x.shape}")
print(f"Data type: {x.dtype}")
print(f"Strides: {x.strides} bytes")
print(f"Total size: {x.nbytes} bytes")
print(f"Is C-contiguous: {x.flags.c_contiguous}")
print(f"Is Fortran-contiguous: {x.flags.f_contiguous}")

Array:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]

Shape: (4, 4)
Data type: int8
Strides: (4, 1) bytes
Total size: 16 bytes
Is C-contiguous: True
Is Fortran-contiguous: False


### Understanding Strides - The Key to Performance

**Strides** tell NumPy how many bytes to skip to move to the next element along each axis.

For our array with `dtype=int8` (1 byte per element):
- To move to next column: skip 1 byte
- To move to next row: skip 4 bytes (entire row)

**Formula**: `element[i,j] = data_ptr + i*stride[0] + j*stride[1]`

In [5]:
# Visualize how strides work
def explain_strides(arr):
    print(f"Array shape: {arr.shape}")
    print(f"Strides: {arr.strides}")
    print(f"Element size: {arr.itemsize} bytes")
    
    # Calculate expected strides
    expected_col_stride = arr.itemsize
    expected_row_stride = arr.shape[1] * arr.itemsize
    print(f"Expected strides: ({expected_row_stride}, {expected_col_stride})")
    
    # Show memory layout
    print("\nMemory layout (conceptual):")
    flat_view = arr.ravel()
    for i in range(len(flat_view)):
        if i % arr.shape[1] == 0 and i > 0:
            print(f"  |  {flat_view[i]:2d}", end="")
        else:
            print(f" {flat_view[i]:2d}", end="")
    print()

explain_strides(x)

Array shape: (4, 4)
Strides: (4, 1)
Element size: 1 bytes
Expected strides: (4, 1)

Memory layout (conceptual):
  0  1  2  3  |   4  5  6  7  |   8  9 10 11  |  12 13 14 15


### Exercise 1: Predict the Strides

For each transformation below, predict the strides before running the code!

In [6]:
# Exercise 1.1: Reshape to (2, 8)
print("Original array (4, 4) with strides", x.strides)
y = x.reshape((2, 8))

print("\n" + "="*50)
print(f"Reshaped to {y.shape}: strides = {y.strides}")
print(y)

Original array (4, 4) with strides (4, 1)

Reshaped to (2, 8): strides = (8, 1)
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]]


In [8]:
# Exercise 1.2: Reshape to (1, 16) 
z = x.reshape((1, 16))
print(f"Reshaped to {z.shape}: strides = {z.strides}")
print(z)

Reshaped to (1, 16): strides = (16, 1)
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]


In [9]:
# Exercise 1.3: Different data type
a = np.array(x, dtype=np.int16)  # 2 bytes per element

print(f"\nWith int16 dtype: strides = {a.strides}")
print(f"Compare with int8: strides = {x.strides}")
print("Notice how strides scale with element size!")


With int16 dtype: strides = (8, 2)
Compare with int8: strides = (4, 1)
Notice how strides scale with element size!


### Views vs Copies: Memory Management Deep Dive

It is important for HPC performance to understand when operations create new memory vs when they just change metadata.

In [8]:
# Demonstrate view vs copy with ravel() and flatten()
x = np.arange(12).reshape(3, 4)
print("Original array:")
print(x)
print(f"Original base: {x.base}")

# ravel() tries to return a view (metadata change only)
y_ravel = x.ravel()
print(f"\nAfter ravel() - same base object? {y_ravel.base is x}")
print(f"Shares memory? {np.shares_memory(x, y_ravel)}")

# flatten() always returns a copy (new memory)
y_flatten = x.flatten()
print(f"\nAfter flatten() - same base object? {y_flatten.base is x}")
print(f"Shares memory? {np.shares_memory(x, y_flatten)}")

Original array:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Original base: [ 0  1  2  3  4  5  6  7  8  9 10 11]

After ravel() - same base object? False
Shares memory? True

After flatten() - same base object? False
Shares memory? False


In [9]:
# Consequence: modifying views affects original
x = np.arange(5)
print(f"Original x: {x}")

y_ravel = x.ravel()  # Creates a view
y_ravel[0] = 999     # Modify the view
print(f"After modifying ravel view: {x}")

# Reset
x = np.arange(5)
y_flatten = x.flatten()  # Creates a copy
y_flatten[0] = 999       # Modify the copy
print(f"After modifying flatten copy: {x}")

Original x: [0 1 2 3 4]
After modifying ravel view: [999   1   2   3   4]
After modifying flatten copy: [0 1 2 3 4]


### Performance Impact: When Views Become Copies

Sometimes operations that normally create views are forced to create copies due to memory layout.

In [10]:
# Create a large array for timing
x = np.random.rand(5000, 5000)
print(f"Array size: {x.nbytes / 1e6:.1f} MB")

# Case 1: Simple operations (fast - metadata only)
%timeit x.T          # Transpose - just swap strides
%timeit x.ravel()    # Ravel of C-contiguous array - view possible

Array size: 200.0 MB
50.8 ns ± 2.84 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
45 ns ± 3.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [11]:
# Case 2: Transpose + ravel (slow - copy required)
%timeit x.T.ravel()  # Can't create view of non-contiguous transpose

# Case 3: Flatten (always slow - always copies)
%timeit x.flatten()  # Always creates copy

57.4 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
30.6 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Key Insight**: The transpose `x.T` creates a view with modified strides, but it's no longer C-contiguous. When we then `ravel()` it, NumPy can't express the flattened result with simple strides, so it must copy the data.

In [12]:
# Verify the contiguity issue
print(f"x.flags.c_contiguous: {x.flags.c_contiguous}")
print(f"x.T.flags.c_contiguous: {x.T.flags.c_contiguous}")
print(f"x.T.flags.f_contiguous: {x.T.flags.f_contiguous}")

# This is why x.T.ravel() is slow - it requires copying to make contiguous

x.flags.c_contiguous: True
x.T.flags.c_contiguous: False
x.T.flags.f_contiguous: True


## 1.3 Array Creation and Basic Operations

### Efficient Array Creation Methods

Different creation methods have different performance characteristics.

In [13]:
# Compare different array creation methods
size = (1000, 1000)

print("Array creation timing:")
%timeit np.zeros(size)           # Pre-allocated, initialized to 0
%timeit np.empty(size)           # Pre-allocated, uninitialized (fastest)
%timeit np.ones(size)            # Pre-allocated, initialized to 1
%timeit np.full(size, 3.14)      # Pre-allocated, initialized to value
%timeit np.arange(size[0] * size[1]).reshape(size)  # Sequential, then reshape

Array creation timing:
183 μs ± 6.44 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
295 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
220 μs ± 20.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
216 μs ± 10.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
236 μs ± 20.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Data Types and Memory Usage

Choosing the right data type can significantly impact memory usage and performance.

In [14]:
# Memory usage comparison for different dtypes
size = 5_000_000
dtypes = [np.int8, np.int16, np.int32, np.int64, np.float32, np.float64]

print("Memory usage by data type:")
print(f"{'Data Type':<12}\t\t{'Bytes/element':<15} {'Total MB':<10}")
print("-" * 50)

base_memory = None
for dtype in dtypes:
    arr = np.ones(size, dtype=dtype)
    memory_mb = arr.nbytes / 1e6
    if base_memory is None:
        base_memory = memory_mb
    
    print(f"{str(dtype):<12}\t{arr.itemsize:<15} {memory_mb:<10.1f}")

Memory usage by data type:
Data Type   		Bytes/element   Total MB  
--------------------------------------------------
<class 'numpy.int8'>	1               5.0       
<class 'numpy.int16'>	2               10.0      
<class 'numpy.int32'>	4               20.0      
<class 'numpy.int64'>	8               40.0      
<class 'numpy.float32'>	4               20.0      
<class 'numpy.float64'>	8               40.0      


### Exercise 2: Memory-Efficient Array Operations

Create arrays with different dtypes and observe the memory and performance implications.

In [15]:
# Exercise: Compare performance of operations on different dtypes
size = 5_000_000

# Create arrays with different precisions
arr_f32 = np.random.rand(size).astype(np.float32)
arr_f64 = np.random.rand(size).astype(np.float64)

print(f"Float32 array: {arr_f32.nbytes / 1e6:.1f} MB")
print(f"Float64 array: {arr_f64.nbytes / 1e6:.1f} MB")

print("\nPerformance comparison for sum operation:")
%timeit np.sum(arr_f32)
%timeit np.sum(arr_f64)

print("\nPerformance comparison for element-wise multiplication:")
%timeit arr_f32 * 2.0
%timeit arr_f64 * 2.0

Float32 array: 20.0 MB
Float64 array: 40.0 MB

Performance comparison for sum operation:
773 μs ± 69.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.55 ms ± 61.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Performance comparison for element-wise multiplication:
1.27 ms ± 55 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
6.15 ms ± 126 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


---

# Session 2: Broadcasting and Vectorization

## 2.1 Understanding Broadcasting - The Heart of NumPy Performance

### Broadcasting Rules

Broadcasting allows NumPy to perform operations on arrays with different shapes efficiently. The rules are:

1. **Start from the trailing dimension** and work backward
2. **Dimensions are compatible** if:
   - They are equal, OR
   - One of them is 1, OR  
   - One of them is missing (treated as 1)
3. **Result shape** is the maximum size along each dimension

```
Examples:
A: (5, 4)     B: (4,)       → Result: (5, 4)  ✓
A: (5, 1)     B: (1, 4)     → Result: (5, 4)  ✓
A: (5, 4)     B: (3,)       → Result: Error   ✗
```

In [16]:
# Basic broadcasting examples
print("Example 1: Array + scalar")
a = np.array([1, 2, 3, 4, 5])
result = a + 10
print(f"[1,2,3,4,5] + 10 = {result}")
print(f"Shapes: {a.shape} + scalar → {result.shape}")

print("\nExample 2: 2D array + 1D array")
b = np.array([[1, 2, 3],
              [4, 5, 6]])
c = np.array([10, 20, 30])
result = b + c
print(f"Array b shape: {b.shape}")
print(f"Array c shape: {c.shape}")
print(f"Result shape: {result.shape}")
print("Result:")
print(result)

Example 1: Array + scalar
[1,2,3,4,5] + 10 = [11 12 13 14 15]
Shapes: (5,) + scalar → (5,)

Example 2: 2D array + 1D array
Array b shape: (2, 3)
Array c shape: (3,)
Result shape: (2, 3)
Result:
[[11 22 33]
 [14 25 36]]


### Creating New Dimensions with `np.newaxis`

Often we need to add dimensions to enable broadcasting. `np.newaxis` is an alias for `None` and adds a dimension of size 1.

In [17]:
# Demonstrate np.newaxis
x = np.arange(5)
print(f"Original x: {x}")
print(f"Shape: {x.shape}")

# Convert to column vector
x_col = x[:, np.newaxis]
print(f"\nColumn vector: \n{x_col}")
print(f"Shape: {x_col.shape}")

# Convert to row vector  
x_row = x[np.newaxis, :]
print(f"\nRow vector: \n{x_row}")
print(f"Shape: {x_row.shape}")

# Verify these are views (no memory copying)
print(f"\nMemory sharing - original and column: {np.shares_memory(x, x_col)}")
print(f"Memory sharing - original and row: {np.shares_memory(x, x_row)}")

Original x: [0 1 2 3 4]
Shape: (5,)

Column vector: 
[[0]
 [1]
 [2]
 [3]
 [4]]
Shape: (5, 1)

Row vector: 
[[0 1 2 3 4]]
Shape: (1, 5)

Memory sharing - original and column: True
Memory sharing - original and row: True


### Exercise 3: Broadcasting Puzzle

Predict the output shapes and results before running!

In [18]:
# Exercise 3.1: Create a multiplication table
x = np.arange(1, 6)  # [1, 2, 3, 4, 5]

print("What will be the shape and result of x[:, newaxis] * x?")

multiplication_table = x[:, np.newaxis] * x
print(f"\nResult shape: {multiplication_table.shape}")
print("Multiplication table:")
print(multiplication_table)

What will be the shape and result of x[:, newaxis] * x?

Result shape: (5, 5)
Multiplication table:
[[ 1  2  3  4  5]
 [ 2  4  6  8 10]
 [ 3  6  9 12 15]
 [ 4  8 12 16 20]
 [ 5 10 15 20 25]]


In [19]:
# Exercise 3.2: 3D broadcasting
a = np.random.rand(2, 3, 1)  # Shape (2, 3, 1)
b = np.random.rand(4)        # Shape (4,)

print(f"Array a shape: {a.shape}")
print(f"Array b shape: {b.shape}")
print("What will be the result shape of a + b?")

result = a + b
print(f"\nActual result shape: {result.shape}")
print("This creates all combinations of elements!")

Array a shape: (2, 3, 1)
Array b shape: (4,)
What will be the result shape of a + b?

Actual result shape: (2, 3, 4)
This creates all combinations of elements!


## 2.2 Advanced Broadcasting Applications

### Pairwise Operations on Vector Collections

A common HPC pattern: given N vectors, compute some pairwise operation between all pairs.

In [20]:
# Generate sample data: 6 vectors of dimension 3
np.random.seed(42)  # For reproducible results
vectors = np.random.rand(6, 3)
print("Input vectors (6 vectors of dimension 3):")
print(vectors)
print(f"Shape: {vectors.shape}")

Input vectors (6 vectors of dimension 3):
[[0.37454012 0.95071431 0.73199394]
 [0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501]
 [0.70807258 0.02058449 0.96990985]
 [0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643]]
Shape: (6, 3)


In [21]:
# Compute all pairwise differences using broadcasting
# Goal: diff[i,j,k] = vectors[i,k] - vectors[j,k]

# Method: Use broadcasting to create (6,1,3) - (1,6,3) → (6,6,3)
pairwise_diff = vectors[:, np.newaxis, :] - vectors[np.newaxis, :, :]

print(f"Pairwise differences shape: {pairwise_diff.shape}")
print("\nExample: Difference between vector 0 and vector 1:")
print(f"Vector 0: {vectors[0]}")
print(f"Vector 1: {vectors[1]}")
print(f"Difference: {pairwise_diff[0, 1]}")
print(f"Manual check: {vectors[0] - vectors[1]}")

# Verify diagonal is zero (vector - itself)
print(f"\nDiagonal elements (should be ~0): {np.diag(pairwise_diff[:,:,0])}")

Pairwise differences shape: (6, 6, 3)

Example: Difference between vector 0 and vector 1:
Vector 0: [0.37454012 0.95071431 0.73199394]
Vector 1: [0.59865848 0.15601864 0.15599452]
Difference: [-0.22411837  0.79469567  0.57599942]
Manual check: [-0.22411837  0.79469567  0.57599942]

Diagonal elements (should be ~0): [0. 0. 0. 0. 0. 0.]


### Memory Usage Analysis

Let's understand the memory implications of this broadcasting operation.

In [22]:
# Analyze memory usage
n_vectors, dim = vectors.shape
input_memory = vectors.nbytes
output_memory = pairwise_diff.nbytes

print(f"Input: {n_vectors} vectors of dimension {dim}")
print(f"Input memory: {input_memory} bytes ({input_memory/1024:.1f} KB)")
print(f"Output memory: {output_memory} bytes ({output_memory/1024:.1f} KB)")
print(f"Memory expansion: {output_memory/input_memory:.1f}x")

print(f"\nFor N={n_vectors} vectors, we create N×N×dim = {n_vectors}×{n_vectors}×{dim} = {n_vectors**2 * dim} elements")
print(f"This scales as O(N²) - can become memory-intensive for large N!")

Input: 6 vectors of dimension 3
Input memory: 144 bytes (0.1 KB)
Output memory: 864 bytes (0.8 KB)
Memory expansion: 6.0x

For N=6 vectors, we create N×N×dim = 6×6×3 = 108 elements
This scales as O(N²) - can become memory-intensive for large N!


## 2.3 Vectorization Strategies

### The Golden Rule: Eliminate Python Loops

Let's see how to replace common loop patterns with vectorized operations.

In [23]:
# Example: Apply a function to each element
data = np.random.rand(100000)

# BAD: Python loop
def apply_function_loop(x):
    result = np.empty_like(x)
    for i in range(len(x)):
        result[i] = np.exp(x[i]) + np.sin(x[i])
    return result

# GOOD: Vectorized
def apply_function_vectorized(x):
    return np.exp(x) + np.sin(x)

# Performance comparison
print("Performance comparison:")
%timeit apply_function_loop(data)
%timeit apply_function_vectorized(data)

# Verify they give the same result
result1 = apply_function_loop(data[:100])  # Use smaller array for speed
result2 = apply_function_vectorized(data[:100])
print(f"\nResults are equal: {np.allclose(result1, result2)}")

Performance comparison:
91.8 ms ± 7.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.03 ms ± 62.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Results are equal: True


### Exercise 4: Vectorization Challenge

Convert the following loop-based operations to vectorized NumPy operations.

In [24]:
# Exercise 4.1: Conditional operations
# Loop version: Set values > 0.5 to their square, others to 0
data = np.random.rand(1000)

def conditional_loop(x):
    result = np.empty_like(x)
    for i in range(len(x)):
        if x[i] > 0.5:
            result[i] = x[i] ** 2
        else:
            result[i] = 0.0
    return result

# TODO: Implement vectorized version
def conditional_vectorized(x):
    # Hint: Use np.where() or boolean indexing
    pass

# Uncomment to test:
#result_loop = conditional_loop(data)
#result_vectorized = conditional_vectorized(data)
#print(f"Results match: {np.allclose(result_loop, result_vectorized)}")

In [25]:
# Solution for Exercise 4.1
def conditional_vectorized(x):
    return np.where(x > 0.5, x**2, 0.0)

# Alternative using boolean indexing
def conditional_vectorized_alt(x):
    result = np.zeros_like(x)
    mask = x > 0.5
    result[mask] = x[mask] ** 2
    return result

# Test and compare performance
result_loop = conditional_loop(data)
result_vec1 = conditional_vectorized(data)
result_vec2 = conditional_vectorized_alt(data)

print(f"Loop vs where(): {np.allclose(result_loop, result_vec1)}")
print(f"Loop vs indexing: {np.allclose(result_loop, result_vec2)}")

print("\nPerformance comparison:")
%timeit conditional_loop(data)
%timeit conditional_vectorized(data)
%timeit conditional_vectorized_alt(data)

Loop vs where(): True
Loop vs indexing: True

Performance comparison:
135 μs ± 8.53 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
2.52 μs ± 72.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
5.84 μs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [26]:
# Exercise 4.2: Cumulative operations
# Convert nested loop to vectorized operation
matrix = np.random.rand(500, 500)

# Loop version: Compute row-wise running averages
def running_average_loop(mat):
    result = np.empty_like(mat)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            result[i, j] = np.mean(mat[i, :j+1])
    return result

# Vectorized version using cumsum
def running_average_vectorized(mat):
    cumsum = np.cumsum(mat, axis=1)
    indices = np.arange(1, mat.shape[1] + 1)
    return cumsum / indices

# Test on smaller matrix for verification
small_mat = matrix[:5, :5]
result_loop = running_average_loop(small_mat)
result_vec = running_average_vectorized(small_mat)

print(f"Results match: {np.allclose(result_loop, result_vec)}")
print("\nPerformance on full matrix:")
%timeit running_average_loop(matrix)
%timeit running_average_vectorized(matrix)

Results match: True

Performance on full matrix:
489 ms ± 8.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
629 μs ± 5.57 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


---

# Session 3: Optimization and Real-World Applications

## 3.1 Euclidean Distance Matrix

### The Problem

Given N vectors in D-dimensional space, compute the matrix of Euclidean distances between all pairs:

$$d_{ij} = \sqrt{\sum_{k=1}^{D} (x_{ik} - x_{jk})^2}$$

This is fundamental in:
- Machine learning (k-NN, clustering)
- Molecular dynamics simulations
- Computer graphics
- Scientific computing

We'll implement and compare multiple approaches, analyzing their trade-offs.

In [27]:
# Generate test data
np.random.seed(42)
n_points = 1000
n_features = 50
X = np.random.rand(n_points, n_features) * 10.0

print(f"Dataset: {n_points} points in {n_features}D space")
print(f"Input size: {X.nbytes / 1e6:.1f} MB")
print(f"Output will be: {n_points}×{n_points} = {n_points**2:,} distances")
print(f"Output size: {(n_points**2 * 8) / 1e6:.1f} MB (float64)")

Dataset: 1000 points in 50D space
Input size: 0.4 MB
Output will be: 1000×1000 = 1,000,000 distances
Output size: 8.0 MB (float64)


### Approach 1: Broadcasting (Intuitive but Memory-Intensive)

The most straightforward approach uses broadcasting to compute all pairwise differences.

In [28]:
def euclidean_broadcast(x, y):
    """
    Euclidean distance matrix using broadcasting.
    
    Args:
        x: (N, D) array of N vectors in D dimensions
        y: (M, D) array of M vectors in D dimensions
        
    Returns:
        (N, M) array of distances
    """
    # Shape: (N, 1, D) - (1, M, D) → (N, M, D)
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
    
    # Sum of squares along last dimension
    return np.sqrt((diff * diff).sum(axis=2))

# Test with small dataset first
X_small = X[:10, :5]
distances_broadcast = euclidean_broadcast(X_small, X_small)

print(f"Small test result shape: {distances_broadcast.shape}")
print("\nFirst few distances:")
print(distances_broadcast[:3, :3])
print("\nDiagonal (should be ~0):")
print(np.diag(distances_broadcast)[:5])

Small test result shape: (10, 10)

First few distances:
[[ 0.          8.43172737  9.81770581]
 [ 8.43172737  0.         12.39060453]
 [ 9.81770581 12.39060453  0.        ]]

Diagonal (should be ~0):
[0. 0. 0. 0. 0.]


### Memory Analysis of Broadcasting Approach

Let's understand the memory requirements:

In [29]:
def analyze_memory_usage(n_points, n_features):
    input_size = n_points * n_features * 8  # float64
    intermediate_size = n_points * n_points * n_features * 8  # diff array
    output_size = n_points * n_points * 8  # distance matrix
    
    print(f"For {n_points} points in {n_features}D:")
    print(f"Input memory:        {input_size / 1e6:8.1f} MB")
    print(f"Intermediate memory: {intermediate_size / 1e6:8.1f} MB (diff array)")
    print(f"Output memory:       {output_size / 1e6:8.1f} MB")
    print(f"Peak memory usage:   {(input_size + intermediate_size + output_size) / 1e6:8.1f} MB")
    print(f"Memory amplification: {intermediate_size / input_size:.1f}x\n")

# Analyze different problem sizes
sizes = [(100, 10), (500, 20), (1000, 50), (2000, 100)]
for n, d in sizes:
    analyze_memory_usage(n, d)

For 100 points in 10D:
Input memory:             0.0 MB
Intermediate memory:      0.8 MB (diff array)
Output memory:            0.1 MB
Peak memory usage:        0.9 MB
Memory amplification: 100.0x

For 500 points in 20D:
Input memory:             0.1 MB
Intermediate memory:     40.0 MB (diff array)
Output memory:            2.0 MB
Peak memory usage:       42.1 MB
Memory amplification: 500.0x

For 1000 points in 50D:
Input memory:             0.4 MB
Intermediate memory:    400.0 MB (diff array)
Output memory:            8.0 MB
Peak memory usage:      408.4 MB
Memory amplification: 1000.0x

For 2000 points in 100D:
Input memory:             1.6 MB
Intermediate memory:   3200.0 MB (diff array)
Output memory:           32.0 MB
Peak memory usage:     3233.6 MB
Memory amplification: 2000.0x



### Approach 2: Mathematical Optimization (The "Trick")

We can avoid the large intermediate array using the algebraic identity:

$$\|x_i - x_j\|^2 = \|x_i\|^2 + \|x_j\|^2 - 2 x_i \cdot x_j$$

This reduces memory usage and can be faster due to optimized BLAS operations.

In [30]:
def euclidean_trick(x, y):
    """
    Euclidean distance matrix using the algebraic trick.
    
    Uses: ||x-y||² = ||x||² + ||y||² - 2⟨x,y⟩
    """
    # Compute squared norms: ||x_i||² for each row
    x_sqnorms = np.einsum('ij,ij->i', x, x)[:, np.newaxis]  # Shape: (N, 1)
    y_sqnorms = np.einsum('ij,ij->i', y, y)[np.newaxis, :]  # Shape: (1, M)
    
    # Compute dot products: x_i · y_j
    xy_dots = x @ y.T  # Shape: (N, M)
    
    # Apply the formula (with abs for numerical stability)
    squared_distances = np.abs(x_sqnorms + y_sqnorms - 2.0 * xy_dots)
    
    return np.sqrt(squared_distances)

# Test and verify it gives same results
distances_trick = euclidean_trick(X_small, X_small)

print(f"Results match: {np.allclose(distances_broadcast, distances_trick, atol=1e-6)}")
print(f"Max difference: {np.abs(distances_broadcast - distances_trick).max():.2e}")

Results match: True
Max difference: 3.37e-07


### Deep Dive: Einstein Summation

`np.einsum` is a powerful tool for expressing tensor operations concisely and efficiently.

In [31]:
# Understanding np.einsum with examples
A = np.random.rand(3, 4)
print(f"Matrix A shape: {A.shape}")

# Example 1: Row-wise sum of squares
method1 = (A * A).sum(axis=1)  # Traditional way
method2 = np.einsum('ij,ij->i', A, A)  # Einstein summation

print(f"Traditional (A*A).sum(axis=1): {method1}")
print(f"Einstein np.einsum('ij,ij->i'): {method2}")
print(f"Results match: {np.allclose(method1, method2)}")

# Example 2: Matrix multiplication
B = np.random.rand(4, 5)
method1 = A @ B
method2 = np.einsum('ik,kj->ij', A, B)

print(f"\nMatrix multiplication A@B vs einsum: {np.allclose(method1, method2)}")
print(f"Max difference: {np.abs(method1 - method2).max():.2e}")

# Performance comparison for row-wise squared norms
large_matrix = np.random.rand(1000, 300)
print("\nPerformance comparison for squared norms:")
%timeit (large_matrix * large_matrix).sum(axis=1)
%timeit np.einsum('ij,ij->i', large_matrix, large_matrix)

Matrix A shape: (3, 4)
Traditional (A*A).sum(axis=1): [1.54320485 0.9392774  1.39845356]
Einstein np.einsum('ij,ij->i'): [1.54320485 0.9392774  1.39845356]
Results match: True

Matrix multiplication A@B vs einsum: True
Max difference: 2.22e-16

Performance comparison for squared norms:
171 μs ± 3.61 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
49.6 μs ± 2.99 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance Comparison: Broadcasting vs Trick

Let's compare both methods across different problem sizes:

In [32]:
# Performance testing on different sizes
test_sizes = [(100, 20), (300, 30), (500, 40)]

for n_points, n_features in test_sizes:
    print(f"\n=== Testing {n_points} points, {n_features} features ===")
    
    # Generate test data
    test_data = np.random.rand(n_points, n_features)
    
    # Memory requirements
    input_mb = test_data.nbytes / 1e6
    output_mb = (n_points**2 * 8) / 1e6
    intermediate_mb = (n_points**2 * n_features * 8) / 1e6
    
    print(f"Memory - Input: {input_mb:.1f} MB, Output: {output_mb:.1f} MB")
    print(f"Broadcasting peak: {input_mb + intermediate_mb + output_mb:.1f} MB")
    print(f"Trick peak: {input_mb + output_mb:.1f} MB")
    
    # Time both methods
    print("Timing broadcast method...")
    time_broadcast = %timeit -o euclidean_broadcast(test_data, test_data)
    
    print("Timing trick method...")
    time_trick = %timeit -o euclidean_trick(test_data, test_data)
    
    speedup = time_broadcast.best / time_trick.best
    print(f"Speedup: {speedup:.2f}x ({time_trick.best:.3f}s vs {time_broadcast.best:.3f}s)")


=== Testing 100 points, 20 features ===
Memory - Input: 0.0 MB, Output: 0.1 MB
Broadcasting peak: 1.7 MB
Trick peak: 0.1 MB
Timing broadcast method...
471 μs ± 41.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Timing trick method...
31.8 μs ± 350 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Speedup: 13.80x (0.000s vs 0.000s)

=== Testing 300 points, 30 features ===
Memory - Input: 0.1 MB, Output: 0.7 MB
Broadcasting peak: 22.4 MB
Trick peak: 0.8 MB
Timing broadcast method...
9.77 ms ± 419 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Timing trick method...
966 μs ± 459 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Speedup: 14.60x (0.001s vs 0.009s)

=== Testing 500 points, 40 features ===
Memory - Input: 0.2 MB, Output: 2.0 MB
Broadcasting peak: 82.2 MB
Trick peak: 2.2 MB
Timing broadcast method...
36.5 ms ± 982 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Timing trick method...
1.65 ms ± 24.6 μs per loop (mean ± std. dev

## 3.2 Performance Profiling and Optimization

### Line-by-Line Profiling

Let's see exactly where time is spent in each function:

In [33]:
# Install line_profiler if not available
try:
    %load_ext line_profiler
except:
    print("line_profiler not available. Install with: pip install line_profiler")
    print("Continuing without detailed profiling...")

In [34]:
# Profile the trick method
profile_data = np.random.rand(800, 40)

try:
    %lprun -f euclidean_trick euclidean_trick(profile_data, profile_data)
except:
    print("Line profiler not available, showing manual timing breakdown:")
    
    # Manual timing breakdown
    import time
    
    # Time each component
    start = time.time()
    x_sqnorms = np.einsum('ij,ij->i', profile_data, profile_data)[:, np.newaxis]
    time1 = time.time() - start
    
    start = time.time()
    y_sqnorms = np.einsum('ij,ij->i', profile_data, profile_data)[np.newaxis, :]
    time2 = time.time() - start
    
    start = time.time()
    xy_dots = profile_data @ profile_data.T
    time3 = time.time() - start
    
    start = time.time()
    result = np.sqrt(np.abs(x_sqnorms + y_sqnorms - 2.0 * xy_dots))
    time4 = time.time() - start
    
    total = time1 + time2 + time3 + time4
    print(f"Compute x squared norms: {time1:.4f}s ({100*time1/total:.1f}%)")
    print(f"Compute y squared norms: {time2:.4f}s ({100*time2/total:.1f}%)")
    print(f"Matrix multiplication:   {time3:.4f}s ({100*time3/total:.1f}%)")
    print(f"Final computation:       {time4:.4f}s ({100*time4/total:.1f}%)")
    print(f"Total:                   {total:.4f}s")

Timer unit: 1e-09 s

Total time: 0.00855432 s
File: /tmp/ipykernel_416929/3430701871.py
Function: euclidean_trick at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_trick(x, y):
     2                                               """
     3                                               Euclidean distance matrix using the algebraic trick.
     4                                           
     5                                               Uses: ||x-y||² = ||x||² + ||y||² - 2⟨x,y⟩
     6                                               """
     7                                               # Compute squared norms: ||x_i||² for each row
     8         1     126858.0 126858.0      1.5      x_sqnorms = np.einsum('ij,ij->i', x, x)[:, np.newaxis]  # Shape: (N, 1)
     9         1      42496.0  42496.0      0.5      y_sqnorms = np.einsum('ij,ij->i', y, y)[np.newaxis, :]  # Shape: (1, M)
    10               

### Understanding NumPy's Multithreading

NumPy uses OpenMP for parallel operations, especially in BLAS routines like matrix multiplication.

In [35]:
# Check BLAS configuration
print("NumPy configuration:")
np.show_config()

# The matrix multiplication x @ y.T is typically the bottleneck
# and benefits most from multiple cores
large_data = np.random.rand(1500, 100)

print("\nTiming matrix multiplication (should use multiple cores):")
%timeit large_data @ large_data.T

NumPy configuration:
Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/_internal/cpython-3.12.7/lib/python3.12/site-packages/scipy_openblas64/include
    lib directory: /opt/_internal/cpython-3.12.7/lib/python3.12/site-packages/scipy_openblas64/lib
    name: scipy-openblas
    openblas configuration: OpenBLAS 0.3.29  USE64BITINT DYNAMIC_ARCH NO_AFFINITY
      Haswell MAX_THREADS=64
    pc file directory: /project/.openblas
    version: 0.3.29
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /opt/_internal/cpython-3.12.7/lib/python3.12/site-packages/scipy_openblas64/include
    lib directory: /opt/_internal/cpython-3.12.7/lib/python3.12/site-packages/scipy_openblas64/lib
    name: scipy-openblas
    openblas configuration: OpenBLAS 0.3.29  USE64BITINT DYNAMIC_ARCH NO_AFFINITY
      Haswell MAX_THREADS=64
    pc file directory: /project/.openblas
    version: 0.3.29
Compilers:
  c:
    commands: cc
    

### Memory Profiling

Let's analyze memory usage patterns:

In [36]:
# Simple memory monitoring
import psutil
import os

def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def memory_profile_function(func, data, name):
    """Profile memory usage of a function"""
    initial_memory = get_memory_usage()
    result = func(data, data)
    peak_memory = get_memory_usage()
    del result  # Free result memory
    final_memory = get_memory_usage()
    
    print(f"{name}:")
    print(f"  Initial: {initial_memory:.1f} MB")
    print(f"  Peak:    {peak_memory:.1f} MB (+{peak_memory-initial_memory:.1f} MB)")
    print(f"  Final:   {final_memory:.1f} MB")
    print()

# Test with moderately sized data
test_data = np.random.rand(800, 50)
print(f"Test data: {test_data.nbytes/1e6:.1f} MB")
print()

memory_profile_function(euclidean_trick, test_data, "Trick method")
memory_profile_function(euclidean_broadcast, test_data, "Broadcast method")

Test data: 0.3 MB

Trick method:
  Initial: 292.1 MB
  Peak:    292.3 MB (+0.2 MB)
  Final:   292.3 MB

Broadcast method:
  Initial: 292.3 MB
  Peak:    292.3 MB (+0.0 MB)
  Final:   292.3 MB



### Exercise 5: Optimization Challenge

Implement an optimized version using `einsum` for the final sum in the broadcast method.

In [37]:
def euclidean_broadcast_optimized(x, y):
    """
    Optimized broadcast version using einsum for the final reduction.
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
    
    # TODO: Use einsum instead of (diff * diff).sum(axis=2)
    squared_distances = np.einsum('ijk,ijk->ij', diff, diff)
    
    return np.sqrt(squared_distances)

# Test the optimized version
test_data = np.random.rand(200, 20)

result_original = euclidean_broadcast(test_data, test_data)
result_optimized = euclidean_broadcast_optimized(test_data, test_data)

print(f"Results match: {np.allclose(result_original, result_optimized)}")

print("\nPerformance comparison:")
%timeit euclidean_broadcast(test_data, test_data)
%timeit euclidean_broadcast_optimized(test_data, test_data)

Results match: True

Performance comparison:
1.84 ms ± 188 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.15 ms ± 96.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## 3.3 Best Practices and HPC Considerations

### When to Use Each Approach

| Method | Memory Usage | Speed | Best When |
|--------|-------------|-------|----------|
| Broadcasting | O(N²D) | Moderate | Small N, large D, simple to understand |
| Algebraic Trick | O(N²) | Fast | Large N, moderate D, memory-limited |
| Chunked Processing | O(chunk_size²D) | Variable | Very large N, limited memory |

### Preparing for Advanced HPC Tools

This NumPy foundation prepares you for:

1. **Numba**: JIT compilation for custom kernels
2. **CuPy**: GPU acceleration with CUDA
3. **Dask**: Distributed and out-of-core computation
4. **MPI**: Multi-node parallelization

In [38]:
# Example: Chunked processing for very large datasets
def euclidean_chunked(x, y, chunk_size=1000):
    """
    Compute distance matrix in chunks to manage memory usage.
    
    This approach trades computation time for memory efficiency.
    """
    n, m = x.shape[0], y.shape[0]
    result = np.empty((n, m), dtype=np.float64)
    
    for i in range(0, n, chunk_size):
        i_end = min(i + chunk_size, n)
        for j in range(0, m, chunk_size):
            j_end = min(j + chunk_size, m)
            
            # Compute distance for this chunk
            chunk_result = euclidean_trick(x[i:i_end], y[j:j_end])
            result[i:i_end, j:j_end] = chunk_result
    
    return result

# Demonstrate chunked processing
large_data = np.random.rand(1200, 30)

print("Comparing chunked vs direct computation:")
print(f"Data size: {large_data.nbytes/1e6:.1f} MB")
print(f"Full result would be: {(1200**2 * 8)/1e6:.1f} MB")

# Time chunked version
%timeit euclidean_chunked(large_data, large_data, chunk_size=400)

# Verify results match (on smaller subset)
subset = large_data[:100]
result_direct = euclidean_trick(subset, subset)
result_chunked = euclidean_chunked(subset, subset, chunk_size=50)
print(f"\nChunked results match direct: {np.allclose(result_direct, result_chunked)}")

Comparing chunked vs direct computation:
Data size: 0.3 MB
Full result would be: 11.5 MB
10.8 ms ± 270 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Chunked results match direct: True


### Common Performance Pitfalls

1. **Creating unnecessary copies**
2. **Using wrong dtypes** (float64 when float32 would suffice)
3. **Not considering memory layout** (C vs Fortran order)
4. **Ignoring broadcasting opportunities**
5. **Mixing NumPy with Python loops**

In [39]:
# Example of dtype impact
data_f64 = np.random.rand(1000, 100).astype(np.float64)
data_f32 = data_f64.astype(np.float32)

print(f"Float64 memory: {data_f64.nbytes/1e6:.1f} MB")
print(f"Float32 memory: {data_f32.nbytes/1e6:.1f} MB")

print("\nFloat64 performance:")
%timeit euclidean_trick(data_f64, data_f64)

print("\nFloat32 performance:")
%timeit euclidean_trick(data_f32, data_f32)

# Check if precision loss is acceptable
result_f64 = euclidean_trick(data_f64[:100], data_f64[:100])
result_f32 = euclidean_trick(data_f32[:100], data_f32[:100])
print(f"\nMax difference: {np.abs(result_f64 - result_f32.astype(np.float64)).max():.2e}")

Float64 memory: 0.8 MB
Float32 memory: 0.4 MB

Float64 performance:
7.16 ms ± 200 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Float32 performance:
3.95 ms ± 498 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Max difference: 5.52e-03


---

# Summary and Next Steps

## What We've Learned

### Core Concepts
1. **NumPy Internals**: Memory layout, strides, views vs copies
2. **Broadcasting**: Efficient operations on arrays with different shapes
3. **Vectorization**: Replacing Python loops with NumPy operations
4. **Performance Optimization**: Multiple approaches to the same problem
5. **Profiling**: Understanding where time and memory are used

### Key Performance Principles
- **Avoid Python loops** at all costs in computational kernels
- **Understand memory patterns** - views are fast, copies are expensive
- **Consider multiple algorithms** - there's often a memory/speed trade-off
- **Profile before optimizing** - measure to find real bottlenecks
- **Choose appropriate data types** - precision vs performance

## Optional Exercise: Design Challenge

Design an algorithm for computing pairwise similarities in a recommender system:
- 100,000 users
- 10,000 items
- Sparse rating matrix (1% filled)
- Memory budget: 8GB

Consider:
1. Data structures and memory layout
2. Algorithmic approaches (full vs approximate)
3. Chunking strategies
4. Opportunities for further optimization

In [49]:
# Final performance demonstration
print("NumPy Performance Showcase")
print("=" * 40)

# Create a substantial computation
n = 5000
X = np.random.rand(n, 50)

print(f"Computing {n}×{n} distance matrix...")
print(f"Total operations: ~{n**2 * 50 / 1e9:.1f} billion")

import time
start_time = time.time()
distances = euclidean_trick(X, X)
end_time = time.time()

elapsed = end_time - start_time
operations_per_sec = (n**2 * 50) / elapsed / 1e9

print(f"Completed in {elapsed:.2f} seconds")
print(f"Performance: {operations_per_sec:.1f} billion ops/second")
print(f"Result matrix: {distances.shape} ({distances.nbytes/1e6:.1f} MB)")
print(f"All diagonal elements ≈ 0: {np.allclose(np.diag(distances), 0, atol=1e-6)}. Max. deviation: {round(np.max(np.diag(distances)), 10)}")


NumPy Performance Showcase
Computing 5000×5000 distance matrix...
Total operations: ~1.2 billion
Completed in 0.31 seconds
Performance: 4.1 billion ops/second
Result matrix: (5000, 5000) (200.0 MB)
All diagonal elements ≈ 0: True. Max. deviation: 1.885e-07
